In [64]:
import pandas as pd
from scipy.stats import nbinom, poisson
import numpy as np


def truncPois_CDF(y, mu):
    f_zero = poisson.pmf(0, mu)
    if y > 0:
        return (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
    else:
        return 0
    
def truncPois_logCDF(y, mu):
    f_zero = poisson.pmf(0, mu)
    if y > 0:
        return np.log((poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero))
    else:
        return 0

In [67]:
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))



def qpois_alt_trunc(p, lam, lower_tail=True, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
        
    if not lower_tail:
        logp = log1mexp(-logp)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles




    #??--------

    # Set log-CDF vector
    lp_max = np.max(nlogp[nlogp < 0])
    upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality
    #------



    


    logcdf = np.array([truncPois_logCDF(yi, lam) for yi in range(1, int(upper)+100)])

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i])
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles

#log.p	logical; if TRUE, probabilities p are given as log(p)
#lower.tail	logical; if TRUE (default), probabilities are P[X≤x], otherwise, P[X>x].
def qpois_alt(p, lam, lower_tail=True, log_p=False):
    # Convert p to array if it's a single value
    if not isinstance(p, (list, np.ndarray)):
        p = np.array([p])
    
    # Set log-probabilities (lower tail)
    n = len(p)
    if log_p:
        logp = p
    else:
        logp = np.log(p)
        
    if not lower_tail:
        logp = log1mexp(-logp)
    
    # Set output and deal with special cases (outputs NA and Inf)
    quantiles = np.full(n, np.nan)
    nna = ~np.isnan(logp)
    nlogp = logp[nna]
    if len(nlogp) == 0:
        return quantiles
    
    quantiles[nna] = np.full(len(nna), np.inf)
    if np.min(nlogp) >= 0:
        return quantiles

    # Set log-CDF vector
    lp_max = np.max(nlogp[nlogp < 0])

    upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max))))
    logcdf = poisson.logcdf(np.arange(upper + 1), lam)

    # Compute output
    for i in range(n):
        if nna[i]:
            if logp[i] < 0:
                quantiles[i] = np.sum(logcdf < logp[i])
    
    # Return output
    if len(quantiles) == 1:
        return quantiles[0]
    else:
        return quantiles





qpois_alt(0.999, 45)




67.0

In [79]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies

#qpois_alt_trunc(quantiles, 200)
#poisson.ppf(quantiles, 200)
qpois_alt(quantiles, 200)

array([158., 161., 162., 164., 165., 165., 166., 167., 167., 168., 168.,
       169., 169., 170., 170., 170., 171., 171., 171., 172., 172., 172.,
       172., 173., 173., 173., 173., 173., 174., 174., 174., 174., 174.,
       175., 175., 175., 175., 175., 175., 176., 176., 176., 176., 176.,
       176., 176., 177., 177., 177., 177., 177., 177., 177., 178., 178.,
       178., 178., 178., 178., 178., 178., 178., 179., 179., 179., 179.,
       179., 179., 179., 179., 179., 180., 180., 180., 180., 180., 180.,
       180., 180., 180., 180., 180., 181., 181., 181., 181., 181., 181.,
       181., 181., 181., 181., 181., 182., 182., 182., 182., 182., 182.,
       182., 182., 182., 182., 182., 182., 182., 183., 183., 183., 183.,
       183., 183., 183., 183., 183., 183., 183., 183., 183., 183., 184.,
       184., 184., 184., 184., 184., 184., 184., 184., 184., 184., 184.,
       184., 184., 184., 185., 185., 185., 185., 185., 185., 185., 185.,
       185., 185., 185., 185., 185., 185., 185., 18

In [63]:
#truncPois_logCDF(np.arange(start= 1, stop =20 + 1), 2)
poisson.logcdf(np.arange(10 + 1), 2)

array([-2.00000000e+00, -9.01387711e-01, -3.90562088e-01, -1.54173310e-01,
       -5.40898509e-02, -1.67023189e-02, -4.54411439e-03, -1.09732080e-03,
       -2.37475523e-04, -4.64991561e-05, -8.30825888e-06])

In [66]:
np.array([truncPois_logCDF(yi, 2) for yi in range(1, int(60)+1)])

array([-1.16143936e+00, -4.68292181e-01, -1.80610109e-01, -6.28270729e-02,
       -1.93419610e-02, -5.25722108e-03, -1.26917990e-03, -2.74649737e-04,
       -5.37772901e-05, -9.60865423e-06, -1.57820275e-06, -2.39800444e-07,
       -3.38925556e-08, -4.47714633e-09, -5.55091750e-10, -6.48349152e-11,
       -7.15771886e-12, -7.49067475e-13, -7.44959650e-14, -7.10542736e-15,
       -6.66133815e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  